In [ ]:
!pip install pynacl
!pip install youtube_dl

import discord
from discord.ext import commands
import youtube_dl
import asyncio


client = commands.Bot(command_prefix ="!", intents=discord.Intents.all())
musics = { }
ytdl = youtube_dl.YoutubeDL()


@client.event
async def on_ready():
    print("Le bot est prÃªt !")

class Video:
    def __init__(self, link):
        video = ytdl.extract_info(link, download=False)
        video_format = video["formats"][0]
        self.url = video["webpage_url"]
        self.stream_url = video_format["url"]

@client.command()
async def leave(ctx):
    client = ctx.guild.voice_client
    await client.disconnect()
    musics[ctx.guild] = []


@client.command()
async def resume(ctx):
    client = ctx.guild.voice_client
    if client.is_paused():
        client.resume()


@client.command()
async def pause(ctx):
    client = ctx.guild.voice_client
    if not client.is_paused():
        client.pause()


@client.command()
async def skip(ctx):
     client = ctx.guild.voice_client
     client.stop()


def play_song(client, queue, song):
    source = discord.PCMVolumeTransformer(discord.FFmpegPCMAudio(song.stream_url 
        , before_options = "-reconnect 1 -reconnect_streamed 1 -reconnect_delay_max 5"))
    
    def next(_):
        if len(queue) > 0:
           new_song = queue[0]
           del queue[0]
           play_song(client, queue, new_song)
        else:
            asyncio.run_coroutine_threadsafe(client.disconnect(), client.loop)

    client.play(source, after=next) 


@client.command()
async def play(ctx, url):
    print("play")
    client = ctx.guild.voice_client

    if client and client.channel:
      video = Video(url)
      musics[ctx.guild].append(video)

    else:
      channel = ctx.author.voice.channel
      video = Video(url)
      musics[ctx.guild] = []
      client = await channel.connect()
      await ctx.send(f"je lance : {video.url}")
      play_song(client, musics[ctx.guild], video)


client.run("MTA0NDg5OTQwNDE4NTg3NDQ2Mg.GuQbZl.VzQJ7bw4SXE-0oWchaLurmZRIenrYzxM4jOMYk")